In [ ]:
import numpy as np

# Define your optimization problem and objectives here

# Initialization
def initialize_population(pop_size, num_variables):
    return np.random.rand(pop_size, num_variables)  # Randomly initialize population

def evaluate_objectives(population):
    # Evaluate the population with respect to all objectives
    # Replace this with your objective functions
    objective1 = np.sum(population, axis=1)  # Example objective 1
    objective2 = np.sum((population - 2) ** 2, axis=1)  # Example objective 2
    return np.column_stack((objective1, objective2))

# Differential Evolution Operators
def mutation(population, F):
    # Mutation operator
    donor_vectors = population[np.random.choice(pop_size, pop_size, replace=False)]
    return population + F * (donor_vectors - population)

def crossover(target_vectors, trial_vectors, CR):
    # Crossover operator
    mask = np.random.rand(pop_size, num_variables) < CR
    return np.where(mask, trial_vectors, target_vectors)

# Multi-Objective Differential Evolution
def multi_objective_differential_evolution(pop_size, num_variables, max_generations, F, CR):
    population = initialize_population(pop_size, num_variables)
    objectives = evaluate_objectives(population)

    for generation in range(max_generations):
        trial_population = mutation(population, F)
        trial_population = np.clip(trial_population, 0, 1)  # Ensure solutions are within bounds
        trial_objectives = evaluate_objectives(trial_population)

        # Survival selection (Non-dominated sorting and crowding distance)
        combined_population = np.vstack((population, trial_population))
        combined_objectives = np.vstack((objectives, trial_objectives))

        # Implement non-dominated sorting and crowding distance here

        # Select the top pop_size solutions based on non-dominated sorting and crowding distance

        # Update population and objectives

        print(f"Generation {generation + 1}/{max_generations} completed.")

    return population, objectives

# Example usage
pop_size = 100
num_variables = 5
max_generations = 100
F = 0.5  # Mutation scaling factor
CR = 0.7  # Crossover probability

final_population, final_objectives = multi_objective_differential_evolution(pop_size, num_variables, max_generations, F, CR)


Generation 1/100 completed.
Generation 2/100 completed.
Generation 3/100 completed.
Generation 4/100 completed.
Generation 5/100 completed.
Generation 6/100 completed.
Generation 7/100 completed.
Generation 8/100 completed.
Generation 9/100 completed.
Generation 10/100 completed.
Generation 11/100 completed.
Generation 12/100 completed.
Generation 13/100 completed.
Generation 14/100 completed.
Generation 15/100 completed.
Generation 16/100 completed.
Generation 17/100 completed.
Generation 18/100 completed.
Generation 19/100 completed.
Generation 20/100 completed.
Generation 21/100 completed.
Generation 22/100 completed.
Generation 23/100 completed.
Generation 24/100 completed.
Generation 25/100 completed.
Generation 26/100 completed.
Generation 27/100 completed.
Generation 28/100 completed.
Generation 29/100 completed.
Generation 30/100 completed.
Generation 31/100 completed.
Generation 32/100 completed.
Generation 33/100 completed.
Generation 34/100 completed.
Generation 35/100 compl

In [ ]:
print(final_population, final_objectives)

[[8.08268861e-01 5.46072949e-01 3.60953581e-02 1.03034454e-01
  1.26004580e-01]
 [3.60166083e-01 2.27417717e-01 4.45456116e-01 8.46953562e-01
  1.20302610e-01]
 [8.21992652e-01 4.99224352e-01 5.05960811e-01 9.69938889e-02
  1.98335922e-01]
 [8.21371852e-01 1.02529577e-01 9.39969415e-01 6.07536845e-02
  5.82405976e-01]
 [9.48626329e-01 6.19704694e-01 6.92915191e-01 9.63659880e-01
  3.20025236e-01]
 [6.11933504e-01 1.18609598e-02 4.52270219e-01 4.27027533e-01
  7.30041586e-01]
 [3.24484970e-01 2.45023589e-01 3.55862710e-01 1.73692812e-01
  7.36602917e-01]
 [5.20543287e-01 2.36233766e-01 7.82389476e-01 2.91821494e-01
  6.85700484e-01]
 [9.27386131e-01 4.52059512e-01 3.55266561e-01 5.04209742e-01
  7.93462219e-01]
 [5.00529040e-01 2.81666711e-01 7.41754451e-01 2.31563581e-01
  8.29422829e-01]
 [8.03376528e-01 6.94430276e-01 8.45061207e-01 5.34623331e-03
  1.54697111e-01]
 [1.99846800e-01 6.59262810e-01 4.54859125e-01 7.16917159e-02
  7.35983397e-01]
 [5.92360368e-02 5.45421442e-01 9.789132

`a multi-objective optimization algorithm like Multi-Objective Differential Evolution (MODE) to optimize the weights of your custom neural network for feature extraction. `





In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Activation
from tensorflow.keras.models import Sequential

class MultiplicationLayer(Layer):
    def __init__(self, num_filters_mult, L1, L2, **kwargs):
        super(MultiplicationLayer, self).__init__(**kwargs)
        self.num_filters_mult = num_filters_mult
        self.L1 = L1
        self.L2 = L2

    def build(self, input_shape):
        self.n = input_shape[2]  # Get the 'n' dimension from the input shape
        self.v = self.add_weight(name='v',
                                 shape=(self.n, 2,self.L1 * self.L2),  # Update the shape to (n, 2, L2)
                                 initializer='random_normal',
                                 trainable=True)
        super(MultiplicationLayer, self).build(input_shape)

    def call(self, inputs):
        # Initialize an empty list to store results for each filter
        results = []

        # Reshape v to match the desired shape
        v_reshaped = tf.reshape(self.v, (self.n, 2, self.L1 * self.L2))

        # Iterate over the multiplication filters (L2)
        for i in range(self.L2):
            # Iterate over the input channels (L1)
            for j in range(self.L1):
                # Perform matrix multiplication between input and v
                channel_result = tf.matmul(inputs[:, :, :, j], v_reshaped[:, :, i * self.L1 + j])

                # Append the channel result to the results list
                results.append(channel_result)

        # Stack the results to form the final result tensor of size [m x 2 x( L1 x L2)]
        result = tf.stack(results, axis=-1)
        #The responses of this layer are then added to obtain a single feature matrix of size m x 2;.
        # Sum the responses along the last axis to obtain a single feature matrix of size m x 2
        result = tf.reduce_sum(result, axis=-1)


        return result


# Step 1: Create MultiObjectiveFeatureExtraction model
def create_mofe_model(input_shape, num_filters_conv, num_filters_mult, L1, L2):
    model = Sequential()
    model.add(Conv2D(num_filters_conv, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MultiplicationLayer(num_filters_mult, L1, L2))
    model.add(Activation('sigmoid'))
    model.add(Flatten())
    return model

# Example usage:
input_shape = (28, 28, 1)  # Adjust the input shape as needed
num_filters_conv = 8
num_filters_mult = 8
L1 = 8
L2 = 8
custom_nn = create_mofe_model(input_shape, num_filters_conv, num_filters_mult, L1, L2)
print(custom_nn.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 8)         208       
                                                                 
 multiplication_layer (Mult  (None, 28, 2)             3584      
 iplicationLayer)                                                
                                                                 
 activation (Activation)     (None, 28, 2)             0         
                                                                 
 flatten (Flatten)           (None, 56)                0         
                                                                 
Total params: 3792 (14.81 KB)
Trainable params: 3792 (14.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_custom_feature_extraction_model(input_shape):
    model = models.Sequential()

    # Convolutional Layer 1
    model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional Layer 2
    model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))


    # Flatten the output
    model.add(layers.Flatten())

    # Dense Layer for feature extraction
    model.add(layers.Dense(256, activation='relu'))

    return model

# Example usage:
input_shape = (28, 28, 1)  # Adjust input shape to match your data
custom_nn = create_custom_feature_extraction_model(input_shape)
print(custom_nn.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 256)              

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
import random

In [ ]:
def evaluate_classifier(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

In [ ]:
def calculate_inter_class_distance(y_true, y_pred):
    num_classes = y_true.shape[1]
    inter_class_distances = []

    for i in range(num_classes):
        for j in range(i + 1, num_classes):
            inter_class_distance_class = tf.reduce_sum(tf.abs(tf.reduce_mean(y_true[:, i], axis=0) - tf.reduce_mean(y_true[:, j], axis=0)))
            inter_class_distances.append(inter_class_distance_class)

    # Calculate inter-class distance using the new formula
    inter_class_distance = tf.reduce_sum(inter_class_distances)
    return inter_class_distance



In [ ]:
def calculate_intra_class_variance(y_true, y_pred):
    num_classes = y_true.shape[1]
    intra_class_variances = []

    for i in range(num_classes):
        # Calculate intra-class variance for each class and feature
        intra_class_variance_class = tf.reduce_sum(tf.math.reduce_variance(y_true[:, i] - y_pred[:, i], axis=0))
        intra_class_variances.append(intra_class_variance_class)

    # Calculate intra-class variance using the new formula
    intra_class_variance = tf.reduce_sum(intra_class_variances)
    return intra_class_variance

In [ ]:
# Defining optimization problem and objectives here
def evaluate_objectives(weights, train_images, train_labels):
    # Create a clone custom neural network using the given weights
    #updated_model = tf.keras.models.clone_model(custom_nn)
    updated_model=custom_nn
        # Extract the current weights from the model
    current_weights = updated_model.get_weights()

    # Initialize a list to store the updated weights
    updated_weights = []

    # Iterate through the layers of the model
    for layer, layer_weights in zip(updated_model.layers, current_weights):
        # Get the shapes of the weight tensors in the layer
        weight_shapes = [w.shape for w in layer.get_weights()]

        # Reshape the weights array according to the weight tensor shapes
        reshaped_weights = []
        weight_index = 0

        for shape in weight_shapes:
            num_params = np.prod(shape)
            reshaped_weight = weights[weight_index : weight_index + num_params].reshape(shape)
            reshaped_weights.append(reshaped_weight)
            weight_index += num_params

        # Append the reshaped weights to the updated_weights list
        updated_weights.extend(reshaped_weights)

    # Set the updated weights in the model
    updated_model.set_weights(updated_weights)


    # Extract features using the custom neural network
    train_features = updated_model.predict(train_images)

    # Calculate inter-class distance and intra-class variance
    inter_class_distance= calculate_inter_class_distance(train_labels, train_features)
    intra_class_variance= calculate_intra_class_variance(train_labels, train_features)

    # Define your objectives:
    # Objective 1: Maximize inter-class distance (Distance should be negated for minimization)
    # Objective 2: Minimize intra-class variance
    objective1 = -inter_class_distance
    objective2 = intra_class_variance

    # You can add more objectives if needed

    return [objective1, objective2]


In [ ]:
# Defining optimization problem and objectives here
def extract_feature_trained_model(weights, train_images, train_labels):
    # Create a clone custom neural network using the given weights
    #updated_model = tf.keras.models.clone_model(custom_nn)
    updated_model=custom_nn
        # Extract the current weights from the model
    current_weights = updated_model.get_weights()

    # Initialize a list to store the updated weights
    updated_weights = []

    # Iterate through the layers of the model
    for layer, layer_weights in zip(updated_model.layers, current_weights):
        # Get the shapes of the weight tensors in the layer
        weight_shapes = [w.shape for w in layer.get_weights()]

        # Reshape the weights array according to the weight tensor shapes
        reshaped_weights = []
        weight_index = 0

        for shape in weight_shapes:
            num_params = np.prod(shape)
            reshaped_weight = weights[weight_index : weight_index + num_params].reshape(shape)
            reshaped_weights.append(reshaped_weight)
            weight_index += num_params

        # Append the reshaped weights to the updated_weights list
        updated_weights.extend(reshaped_weights)

    # Set the updated weights in the model
    updated_model.set_weights(updated_weights)


    # Extract features using the custom neural network
    train_features = updated_model.predict(train_images)



    # You can add more objectives if needed

    return train_features


In [ ]:
# Initialize the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
input_shape = (28, 28, 1)  # Adjust the input shape as needed

# Preprocess the dataset
train_images = train_images.astype('float32') / 255.0
train_images = np.expand_dims(train_images, axis=-1)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)

In [ ]:
# Extract features using the custom neural network
train_features = custom_nn.predict(train_images)

# Calculate inter-class distance and intra-class variance
inter_class_distance= calculate_inter_class_distance(train_labels, train_features)
intra_class_variance= calculate_intra_class_variance(train_labels, train_features)

1875/1875 [==============================] - 3s 1ms/step


In [ ]:
inter_class_distance

<tf.Tensor: shape=(), dtype=float32, numpy=0.27326667>

In [ ]:
intra_class_variance

<tf.Tensor: shape=(), dtype=float32, numpy=0.9123746>

In [ ]:
# Initialization
def initialize_population(pop_size, num_variables):
    return np.random.rand(pop_size, num_variables)  # Randomly initialize population


In [ ]:
import numpy as np

def non_dominated_sort(objectives):
    num_solutions = len(objectives)
    domination_count = np.zeros(num_solutions, dtype=int)
    dominated_solutions = [[] for _ in range(num_solutions)]
    frontiers = []
    ranks = np.zeros(num_solutions, dtype=int)

    for i in range(num_solutions):
        for j in range(i + 1, num_solutions):
            if dominates(objectives[i], objectives[j]):
                dominated_solutions[i].append(j)
                domination_count[j] += 1
            elif dominates(objectives[j], objectives[i]):
                dominated_solutions[j].append(i)
                domination_count[i] += 1

        if domination_count[i] == 0:
            ranks[i] = 0
            frontiers.append(i)

    current_rank = 0
    while frontiers:
        next_frontiers = []
        for i in frontiers:
            for j in dominated_solutions[i]:
                domination_count[j] -= 1
                if domination_count[j] == 0:
                    ranks[j] = current_rank + 1
                    next_frontiers.append(j)
        current_rank += 1
        frontiers = next_frontiers

    return ranks

def dominates(obj1, obj2):
    # Returns True if obj1 dominates obj2, i.e., obj1 is better than obj2 in all objectives
    return all(o1 <= o2 for o1, o2 in zip(obj1, obj2))

def calculate_crowding_distance(objectives, front, num_objectives):
    num_solutions = len(front)
    crowding_distance = np.zeros(num_solutions)

    for m in range(num_objectives):
        front_sorted = np.argsort(objectives[front, m])
        crowding_distance[front_sorted[0]] = crowding_distance[front_sorted[-1]] = np.inf
        f_min = objectives[front[front_sorted[0]], m]
        f_max = objectives[front[front_sorted[-1]], m]

        if f_max == f_min:
            continue

        for i in range(1, num_solutions - 1):
            crowding_distance[front_sorted[i]] += (objectives[front[front_sorted[i + 1]], m] -
                                                   objectives[front[front_sorted[i - 1]], m]) / (f_max - f_min)

    return crowding_distance

def select_top_solutions(objectives, ranks, pop_size):
    num_solutions = len(objectives)
    num_objectives = len(objectives[0])

    # Create a list of fronts, where each front contains solutions of the same rank
    fronts = [[] for _ in range(int(np.max(ranks)) + 1)]
    for i in range(num_solutions):
        fronts[int(ranks[i])].append(i)

    selected_population = []
    num_selected = 0
    current_front = 0

    while num_selected < pop_size:
        if len(fronts[current_front]) + num_selected <= pop_size:
            # If adding the entire front doesn't exceed pop_size, add it
            selected_population.extend(fronts[current_front])
            num_selected += len(fronts[current_front])
            current_front += 1
        else:
            # Sort the solutions in the current front based on crowding distance
            front_objectives = np.array([objectives[i] for i in fronts[current_front]])
            crowding_distance = calculate_crowding_distance(front_objectives, fronts[current_front], num_objectives)
            crowding_order = np.argsort(crowding_distance)[::-1]
            selected_population.extend([fronts[current_front][i] for i in crowding_order][:pop_size - num_selected])
            break

    return selected_population[:pop_size]  # Ensure the selected population size is exactly pop_size

# Example usage:
# Define your objectives as a list of lists, where each inner list represents objectives for a solution
objectives = [[1, 2], [3, 4], [2, 3], [4, 1], [3, 3]]
ranks = non_dominated_sort(objectives)
print("Ranks:", ranks)
selected_population = select_top_solutions(objectives, ranks, 3)
print("Selected Population:", selected_population)


Ranks: [0 3 1 0 2]
Selected Population: [0, 3, 2]


In [ ]:
# Differential Evolution Operators
def mutation(population, F):
    # Mutation operator
    pop_size = len(population)  # Get the population size

    indices = list(range(pop_size))
    random.shuffle(indices)
    donor_vectors = [population[i] for i in indices]

    # Convert the population and donor_vectors to numpy arrays for element-wise operations
    population = np.array(population)
    donor_vectors = np.array(donor_vectors)

    return list(population + F * (donor_vectors - population))








def crossover(target_vectors, trial_vectors, CR):
    # Crossover operator
    mask = np.random.rand(pop_size, num_variables) < CR
    return np.where(mask, trial_vectors, target_vectors)

# Multi-Objective Differential Evolution
def multi_objective_differential_evolution(pop_size, num_variables, max_generations, F, CR, train_images, train_labels):
    population = initialize_population(pop_size, num_variables)
    objectives = []  # Initialize an empty list to store objectives

    for generation in range(max_generations):
        print(f"{generation} -----------------------------")
        trial_population = mutation(population, F)
        trial_population = np.clip(trial_population, 0, 1)  # Ensure solutions are within bounds
        print(trial_population)

        # Evaluate objectives for trial population
        trial_objectives = [evaluate_objectives(weights, train_images, train_labels) for weights in trial_population]


        # Implement non-dominated sorting and crowding distance
        ranks = non_dominated_sort(trial_objectives)
        top_solutions = select_top_solutions( trial_objectives, ranks, pop_size)

        # Update population and objectives with the selected solutions
        #population = top_solutions
        population = [population[i] for i in top_solutions]

        objectives = trial_objectives

        print(f"Generation {generation + 1}/{max_generations} completed.")

    return population, objectives


In [ ]:

# Calculate the number of trainable parameters (weights) in the model
num_trainable_weights = custom_nn.count_params()

print("Number of trainable weights:", num_trainable_weights)

# Example usage
pop_size = 4
num_variables = num_trainable_weights  # Use the actual number of weights in your model

print("Number of trainable weights:", num_variables)
max_generations = 5
F = 0.5  # Mutation scaling factor
CR = 0.7  # Crossover probability

# Call the multi-objective differential evolution function to optimize the weights
final_population, final_objectives = multi_objective_differential_evolution(pop_size, num_variables, max_generations, F, CR,train_images, train_labels)


Number of trainable weights: 103904
Number of trainable weights: 103904
0 -----------------------------
[[0.73418722 0.59751408 0.05948286 ... 0.2847123  0.91014355 0.69047591]
 [0.09007344 0.76071731 0.47471063 ... 0.06829643 0.77406716 0.75434077]
 [0.76649592 0.39422821 0.57699122 ... 0.45475215 0.4568856  0.90251626]
 [0.76649592 0.39422821 0.57699122 ... 0.45475215 0.4568856  0.90251626]]
1875/1875 [==============================] - 3s 1ms/step
Generation 1/5 completed.
1 -----------------------------
[[0.09007344 0.76071731 0.47471063 ... 0.06829643 0.77406716 0.75434077]
 [0.8324886  0.45401603 0.36294784 ... 0.56720237 0.78409385 0.76637804]
 [0.8324886  0.45401603 0.36294784 ... 0.56720237 0.78409385 0.76637804]
 [0.60220187 0.47793844 0.48756961 ... 0.05981187 0.25572705 0.96275235]]
1875/1875 [==============================] - 3s 2ms/step
Generation 2/5 completed.
2 -----------------------------
[[0.51043171 0.53561764 0.57056173 ... 0.45899443 0.71605565 0.79831047]
 [0.346

In [ ]:
#final_trial_population = mutation(final_population, F)
final_trial_population=np.array(final_population)
final_trial_population = np.clip(final_trial_population, 0, 1)  # Ensure solutions are within bounds
print(final_trial_population)

[[0.09007344 0.76071731 0.47471063 ... 0.06829643 0.77406716 0.75434077]
 [0.60220187 0.47793844 0.48756961 ... 0.05981187 0.25572705 0.96275235]
 [0.73418722 0.59751408 0.05948286 ... 0.2847123  0.91014355 0.69047591]
 [0.93078998 0.31051798 0.66641283 ... 0.84969243 0.65804414 0.84228017]]


In [ ]:

train_features_list=[]
for weights in final_trial_population:
    updated_model = tf.keras.models.clone_model(custom_nn)
        # Extract the current weights from the model
    current_weights = updated_model.get_weights()

    # Initialize a list to store the updated weights
    updated_weights = []

    # Iterate through the layers of the model
    for layer, layer_weights in zip(updated_model.layers, current_weights):
        # Get the shapes of the weight tensors in the layer
        weight_shapes = [w.shape for w in layer.get_weights()]

        # Reshape the weights array according to the weight tensor shapes
        reshaped_weights = []
        weight_index = 0

        for shape in weight_shapes:
            num_params = np.prod(shape)
            reshaped_weight = weights[weight_index : weight_index + num_params].reshape(shape)
            reshaped_weights.append(reshaped_weight)
            weight_index += num_params

        # Append the reshaped weights to the updated_weights list
        updated_weights.extend(reshaped_weights)

    # Set the updated weights in the model
    updated_model.set_weights(updated_weights)


    # Extract features using the custom neural network
    train_features = updated_model.predict(train_images)
    train_features_list.append(train_features)



1875/1875 [==============================] - 3s 1ms/step


In [ ]:
# Assuming you have final_population and final_objectives

# Calculate the Pareto front ranks using your non-dominated sorting function
pareto_ranks = non_dominated_sort(final_objectives)

# Find the solutions that belong to the Pareto front (rank 1)
pareto_front_indices = [i for i, rank in enumerate(pareto_ranks) if rank == 1]

# Extract the weights corresponding to the Pareto front
pareto_front_weights = [final_trial_population[i] for i in pareto_front_indices]


In [ ]:
pareto_front_weights

[array([0.09667228, 0.9518865 , 0.47721492, ..., 0.56211838, 0.96088953,
        0.38476003])]

In [ ]:
#updated_model = tf.keras.models.clone_model(custom_nn)
        # Extract the current weights from the model
updated_model=custom_nn
current_weights = updated_model.get_weights()

    # Initialize a list to store the updated weights
updated_weights = []

    # Iterate through the layers of the model
for layer, layer_weights in zip(updated_model.layers, current_weights):
        # Get the shapes of the weight tensors in the layer
    weight_shapes = [w.shape for w in layer.get_weights()]

        # Reshape the weights array according to the weight tensor shapes
    reshaped_weights = []
    weight_index = 0

    for shape in weight_shapes:
        num_params = np.prod(shape)
        reshaped_weight = pareto_front_weights[weight_index : weight_index + num_params].reshape(shape)
        reshaped_weights.append(reshaped_weight)
        weight_index += num_params

        # Append the reshaped weights to the updated_weights list
    updated_weights.extend(reshaped_weights)

    # Set the updated weights in the model
updated_model.set_weights(updated_weights)


    # Extract features using the custom neural network
train_features = updated_model.predict(train_images)

AttributeError: ignored

In [ ]:
train_features[100]

array([7777119. , 7997702. , 7220512.5, 8006653.5, 7720430. , 8073538. ,
       8144563. , 8232164.5, 8176256.5, 7543603. , 7255193. , 7627174.5,
       7575721. , 7933093.5, 8163691. , 7251688.5, 7729919.5, 7674822. ,
       8309133. , 7207106. , 8244651.5, 7434572.5, 8138734. , 7924461. ,
       7854826.5, 7732032.5, 7712145.5, 7213845.5, 7469035.5, 8298029. ,
       7456403.5, 7588150.5, 7678270. , 7724569. , 7830335.5, 7269918. ,
       7693473. , 7325042.5, 7811394.5, 7426039.5, 7052945. , 7790045.5,
       7715797. , 7224350.5, 7612287.5, 7595324.5, 7418945.5, 6744634. ,
       7949279. , 8170516.5, 7611815.5, 7632948.5, 7532130. , 8049998.5,
       7654637.5, 7970362.5, 7516620. , 8314517.5, 8154578.5, 7820690.5,
       8052119. , 7298562. , 7680664. , 7988878.5, 8517596. , 7800164. ,
       7514580.5, 6768123.5, 7872916. , 7628923. , 8124736. , 7865111. ,
       7616682. , 8002407. , 7398432.5, 8018936. , 7831197. , 7901632.5,
       8138092.5, 6970833.5, 7666737. , 7582867. , 

In [ ]:
for features in train_features_list:
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=5)
    classifier.fit(features, train_labels)

    # Evaluate the accuracy of the classifier on the training data
    predicted_labels = classifier.predict(features)
    accuracy = accuracy_score(train_labels, predicted_labels)
    print(f"Classification Accuracy (train): {accuracy:.2f}")

Classification Accuracy (train): 0.87
Classification Accuracy (train): 0.87
Classification Accuracy (train): 0.87
Classification Accuracy (train): 0.87


In [ ]:
train_features[200]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.], dtype=float32)

(1, 3792)